<a href="https://colab.research.google.com/github/Atirtacx/Spell_Checking_Bahasa_Indonesia/blob/main/Spell_Checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time

def lev_distance(word1, word2):
    n, m = len(word1), len(word2)
    if n > m:
        word1, word2 = word2, word1
        n, m = m, n

    current_row = range(n + 1)
    for i in range(1, m + 1):
        previous_row, current_row = current_row, [i] + [0] * n
        for j in range(1, n + 1):
            add, delete, change = previous_row[j] + 1, current_row[j-1] + 1, previous_row[j-1]
            if word1[j-1] != word2[i-1]:
                change += 1
            current_row[j] = min(add, delete, change)

    return current_row[n]

def spell_checker(word, dataset):
    min_distance = len(word)
    suggestion = ""

    for correct_word in dataset:
        distance = lev_distance(word, correct_word)
        if distance < min_distance:
            min_distance = distance
            suggestion = correct_word
        elif distance == min_distance:
            suggestion += ", " + correct_word
    
    return suggestion

# Main program
if __name__ == '__main__':
    start = time.time()
    # Load dataset
    with open('kata-dasar.txt', 'r', encoding='utf-8') as f:
        dataset = f.read().splitlines()

    # Test spell checker
    word = input("Masukkan kata yang akan diperiksa: ")
    suggestion = spell_checker(word, dataset)
    print("Kata yang Anda masukkan: ", word)
    print("Kata yang tepat: ", suggestion)
    end = time.time()
    print("Runtime: ", end - start, "detik")

Masukkan kata yang akan diperiksa: sya
Kata yang Anda masukkan:  sya
Kata yang tepat:  isya, iya, saya, sia, sua, swa, syah, syak, syal, syam, syar
Runtime:  3.55381441116333 detik


In [10]:
#Lavenshtein_Distance
import time

def load_words(file_path):
    with open(file_path, 'r') as f:
        words = f.read().splitlines()
    return words

def levenshtein_distance(s, t):
    m = len(s)
    n = len(t)
    d = [[0] * (n + 1) for _ in range(m + 1)]

    for i in range(1, m + 1):
        d[i][0] = i

    for j in range(1, n + 1):
        d[0][j] = j

    for j in range(1, n + 1):
        for i in range(1, m + 1):
            if s[i - 1] == t[j - 1]:
                d[i][j] = d[i - 1][j - 1]
            else:
                d[i][j] = min(d[i - 1][j] + 1,  # deletion
                              d[i][j - 1] + 1,  # insertion
                              d[i - 1][j - 1] + 1)  # substitution

    return d[m][n]

# fungsi spell checker dengan spell suggestion dan spell correction
def spell_checker(word, dictionary):
    min_distance = float('inf')
    closest_word = ''
    suggestions = []
    for dict_word in dictionary:
        distance = levenshtein_distance(word, dict_word)
        if distance < min_distance:
            min_distance = distance
            closest_word = dict_word
        if distance <= 1: # atur threshold jarak terdekat
            suggestions.append(dict_word)
            
    if min_distance == 0:
        return word
    elif len(suggestions) > 0:
        return f"Kata yang dimaksud mungkin adalah: {', '.join(suggestions)}"
    else:
        return f"Tidak ditemukan kata yang cocok. Kata yang dimaksud mungkin adalah: {closest_word}"

# load dictionary
start_time = time.time()
dictionary = load_words('kata-dasar.txt')
print(f"Dictionary loaded in {time.time()-start_time} seconds.")

# test spell checker
words_to_test = ['kemaren', 
                 'ku', 
                 'knalpot', 
                 'brangkas', 
                 'biskuut', 
                 'kemeja', 
                 'selasa', 
                 'kmana', 
                 'kenap', 
                 'siap']

for word in words_to_test:
    start_time = time.time()
    suggestion = spell_checker(word, dictionary)
    print(f"{word} -> {suggestion} (Time taken: {time.time()-start_time} seconds)")


Dictionary loaded in 0.00421905517578125 seconds.
kemaren -> Kata yang dimaksud mungkin adalah: kemarin (Time taken: 0.8033859729766846 seconds)
ku -> Kata yang dimaksud mungkin adalah: aku, kau, kiu, kru, kue, kui, kuk, kup, kur, kus, mu (Time taken: 0.2973766326904297 seconds)
knalpot -> knalpot (Time taken: 0.8307244777679443 seconds)
brangkas -> Kata yang dimaksud mungkin adalah: bangkas, brankas (Time taken: 1.7683281898498535 seconds)
biskuut -> Kata yang dimaksud mungkin adalah: biskuit (Time taken: 1.4422619342803955 seconds)
kemeja -> kemeja (Time taken: 0.6912298202514648 seconds)
selasa -> selasa (Time taken: 0.683840274810791 seconds)
kmana -> Kata yang dimaksud mungkin adalah: kana, koana, mana (Time taken: 0.5888636112213135 seconds)
kenap -> kenap (Time taken: 0.5935609340667725 seconds)
siap -> siap (Time taken: 0.4909698963165283 seconds)


In [11]:
#Peter_Norvig

import re
from collections import Counter

def words(text):
    return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('kata-dasar.txt').read()))

def P(word, N=sum(WORDS.values())):
    "Probability of `word`."
    return WORDS[word] / N

def correction(word):
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def suggestion_word(word):
    "Return suggestions for misspelled word"
    return sorted(candidates(word), key=P, reverse=True)

# load dictionary
start_time = time.time()
dictionary = load_words('kata-dasar.txt')
print(f"Dictionary loaded in {time.time()-start_time} seconds.")

# Testing
test_words = ['kemaren', 
                 'ku', 
                 'knalpot', 
                 'brangkas', 
                 'biskuut', 
                 'kemeja', 
                 'selasa', 
                 'kemna', 
                 'minm', 
                 'siap']
for word in test_words:
    start = time.time()
    corrected_word = correction(word)
    suggestions = suggestion_word(word)
    end = time.time()
    print(f"{word} -> {corrected_word}, suggestions: {suggestions}, runtime: {end-start:.6f} seconds")


Dictionary loaded in 0.003072500228881836 seconds.
kemaren -> kemarin, suggestions: ['kemarin'], runtime: 0.000448 seconds
ku -> kup, suggestions: ['kup', 'kus', 'kiu', 'kue', 'kui', 'aku', 'kau', 'kur', 'kuk', 'mu', 'kru'], runtime: 0.000141 seconds
knalpot -> knalpot, suggestions: ['knalpot'], runtime: 0.000008 seconds
brangkas -> brankas, suggestions: ['brankas', 'bangkas'], runtime: 0.000405 seconds
biskuut -> biskuit, suggestions: ['biskuit'], runtime: 0.000326 seconds
kemeja -> kemeja, suggestions: ['kemeja'], runtime: 0.000007 seconds
selasa -> selasa, suggestions: ['selasa'], runtime: 0.000007 seconds
kemna -> keman, suggestions: ['keman', 'kempa', 'kena'], runtime: 0.000250 seconds
minm -> mim, suggestions: ['mim', 'minum', 'mini', 'minim', 'min', 'mina'], runtime: 0.000190 seconds
siap -> siap, suggestions: ['siap'], runtime: 0.000009 seconds


In [24]:
import time

# Load dictionary for Levenshtein Distance Spell Checker
dictionary = []
with open("kata-dasar.txt", "r", encoding="utf-8") as f:
    for line in f:
        dictionary.append(line.strip())

# Define Levenshtein Distance function
def levenshtein_distance(word1, word2):
    m = len(word1)
    n = len(word2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if word1[i - 1] == word2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = 1 + min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1])
    return dp[m][n]

# Define Levenshtein Distance Spell Checker function
def levenshtein_spell_checker(word):
    min_distance = float('inf')
    corrected_word = word
    for w in dictionary:
        distance = levenshtein_distance(word, w)
        if distance < min_distance:
            min_distance = distance
            corrected_word = w
    return corrected_word

# Define Peter Norvig Spell Checker function
def norvig_spell_checker(word):
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    def edit_distance_1(word):
        splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes    = [a + b[1:] for a, b in splits if b]
        transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b) > 1]
        replaces   = [a + c + b[1:] for a, b in splits for c in alphabet if b]
        inserts    = [a + c + b     for a, b in splits for c in alphabet]
        return set(deletes + transposes + replaces + inserts)

    def edit_distance_2(word):
        return (e2 for e1 in edit_distance_1(word) for e2 in edit_distance_1(e1))

    def known_words(words):
        return set(w for w in words if w in dictionary_frequency)

    def word_probability(word):
        return dictionary_frequency[word]

    suggestions = []
    # Get suggestions based on edit distance of 1
    edit_dist_1 = edit_distance_1(word)
    suggestions += known_words(edit_dist_1)
    # Get suggestions based on edit distance of 2
    edit_dist_2 = edit_distance_2(word)
    suggestions += known_words(edit_dist_2)
    if suggestions:
        return max(suggestions, key=word_probability)
    else:
        return word

# Define Combined Spell Checker function
def combined_spell_checker(word):
    norvig_corrected_word = norvig_spell_checker(word)
    if norvig_corrected_word == word:
        return levenshtein_spell_checker(word)
    else:
        return norvig_corrected_word

sentences = [    "saya ingin membeli bku",    
             "kami sedang belajar bahasa pinisil",    
             "tolong dapaatkan buku teresebut",    
             "buku nya bgaus",    
             "teman saya sdang sakit",    
             "hari ini saya akn pulang larut",    
             "apkah kamu tahu dmana saya bisa membeli baju ini",    
             "sya sduah mnum obat",    
             "di mna kunci nya?",   
             "kucing saya lari kmarin",    
             "saya ingin membeli laptop bagus dengan hrga terjangkau",    
             "apkah kamu bisa membantu sya?",    
             "tlpnonnya sblmnya tlah sya hpus",    
             "saya ingin membeli hp baru",    
             "tolong krmkan buku itu ke alamat saya",   
             "saya akan berangkat ke bandara jam 8 pagi",    
             "tolong antar saya ke alamat ini",    
             "sya tdk bisa dtng hr ini",    
             "maaf saya trlambat",    
             "kemana kamu akan prgi mlm ini?"]

norvig_runtime = 0
levenshtein_runtime = 0
combined_runtime = 0

for sentence in sentences:
    start = time.time()
    norvig_corrected_sentence = ' '.join([norvig_spell_checker(word) for word in sentence.split()])
    end = time.time()
    norvig_runtime += (end - start)
    
    start = time.time()
    levenshtein_corrected_sentence = ' '.join([levenshtein_spell_checker(word) for word in sentence.split()])
    end = time.time()
    levenshtein_runtime += (end - start)
    
    start = time.time()
    combined_corrected_sentence = ' '.join([combined_spell_checker(word) for word in sentence.split()])
    end = time.time()
    combined_runtime += (end - start)

    print("Original sentence:", sentence)
    print("Norvig corrected sentence:", norvig_corrected_sentence)
    print("Levenshtein corrected sentence:", levenshtein_corrected_sentence)
    print("Combined corrected sentence:", combined_corrected_sentence)
    print("----------------------")

print("Norvig runtime:", norvig_runtime)
print("Levenshtein runtime:", levenshtein_runtime)
print("Combined runtime:", combined_runtime)



Original sentence: saya ingin membeli bku
Norvig corrected sentence: saya ingin membeli bku
Levenshtein corrected sentence: saya ingin embel aku
Combined corrected sentence: saya ingin embel aku
----------------------
Original sentence: kami sedang belajar bahasa pinisil
Norvig corrected sentence: kami sedang belajar bahasa pinisil
Levenshtein corrected sentence: kami sedang bajar bahasa pinisi
Combined corrected sentence: kami sedang bajar bahasa pinisi
----------------------
Original sentence: tolong dapaatkan buku teresebut
Norvig corrected sentence: tolong dapaatkan buku teresebut
Levenshtein corrected sentence: tolong adakan buku terbut
Combined corrected sentence: tolong adakan buku terbut
----------------------
Original sentence: buku nya bgaus
Norvig corrected sentence: buku nya bgaus
Levenshtein corrected sentence: buku iya agas
Combined corrected sentence: buku iya agas
----------------------
Original sentence: teman saya sdang sakit
Norvig corrected sentence: teman saya sdan